In [ ]:
#import dependencies
import os
import json
from nltk.corpus.reader import twitter
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import nltk
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords, twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re, string, random
from nltk import FreqDist, classify, NaiveBayesClassifier
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProjectBCS").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Output File (CSV)
output_data_file = "./output_data/words.csv"

# response =  urllib.request.urlopen('https://data-bootcamp-036.s3.us-east-2.amazonaws.com/tweets.csv')

# create a connection to Postgres
conn = psycopg2.connect(database="postgres", user='root', password='******', host='******.us-east-2.rds.amazonaws.com', port='5431')
conn.autocommit = True
cursor = conn.cursor()

# create the database connection
engine = create_engine(f'postgresql://root:******@*******.us-east-2.rds.amazonaws.com:5431/twitter_db')

response = urllib.request.urlopen(engine)
html = response.read()
# print(html)

soup = BeautifulSoup(html,'html5lib')
text = soup.get_text(strip = True)
# print(text)

from nltk.tokenize import TweetTokenizer
tokened = TweetTokenizer()
tokens = tokened.tokenize(text)




In [ ]:
def remove_noise(tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens
    # print(cleaned_tokens)




In [ ]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token



In [ ]:
def get_tweets_for_model(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tokens)



In [ ]:
if __name__ == "__main__":

    # positive_tweets = twitter_samples.strings('positive_tweets.json')
    # negative_tweets = twitter_samples.strings('negative_tweets.json')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    stop_words = stopwords.words('english')

    

In [ ]:
    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []
 
    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    # all_pos_words = get_all_words(cleaned_tokens_list)

    freq_dist_pos = FreqDist(tokens)
    # print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
    cleaned_tokens_for_model = get_tweets_for_model(cleaned_tokens_list=tokens)

    positive_dataset = [(tweet_dict, "Positive")
                        for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                        for tweet_dict in negative_tokens_for_model]

    undefined_dataset = [(tweet_dict, "undefined")
                        for tweet_dict in cleaned_tokens_for_model]

    dataset = positive_dataset + negative_dataset
    testset = undefined_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = testset[:7000]

    classifier = NaiveBayesClassifier.train(train_data)

    # print("Accuracy is:", classify.accuracy(classifier, test_data))
    # print(classifier.show_most_informative_features(10))
    # print(classifier.classify(dict([token, True] for token in tokens)))
    # print(tweet_tokens[0])
    # print(pos_tag(tweet_tokens[0]))
    # print(remove_noise(tokens, stop_words))
    # print(positive_tweet_tokens[500])
    # print(positive_cleaned_tokens_list[500])

    freq = nltk.FreqDist(clean_tokens)
for key,val in freq.items():
    print(str(key) + ':' + str(val))
freq.plot(20, cumulative=False)


df = pd.DataFrame(clean_tokens)

df.to_csv(output_data_file)